# Playground

In [2]:
import keras
from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Model, Sequential
from keras import backend as K
from keras.layers.advanced_activations import PReLU

def prelu_init(shape, dtype=None):
    return -1
import time

import numpy as np

from forwardThinking.datasets import load_mnist
from forwardThinking.models import WeddingCake

Using TensorFlow backend.


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
x_train, y_train, x_test, y_test = load_mnist()
input_dim = 784
hidden_dim = 100
output_dim = 10

In [5]:
model = Sequential()
model.add(Dense(hidden_dim, input_shape=(input_dim,), name='dense_0'))
model.add(Dense(output_dim, name='output_dense'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
h = model.fit(x_train, y_train, epochs=2, verbose=False, validation_data=(x_test, y_test))

In [7]:
def extend(model, hidden_dim, output_dim):
    """ Add a hidden layer to the model. """
    
    # Create model
    new_model = Sequential()
    for i, l in enumerate(model.layers[:-1]):
        if "dense" in l.name:
            new_model.add(Dense(l.output_shape[1], input_shape=(l.input_shape[1],), name="dense_{}".format(i)))
        elif "activation" in l.name:
            new_model.add(Dense(l.output_shape[1], input_shape=(l.input_shape[1],), name="dense_{}".format(i)))
        
    # new layer
    new_model.add(Dense(hidden_dim, activation='linear',
                        kernel_initializer = keras.initializers.Identity(gain=1.0), 
                        bias_initializer='zeros', name="dense_{}".format(i+1)))
    new_model.add(PReLU(alpha_initializer=prelu_init, name='activation_{}'.format(i+1)))
    
    # same output layer
    old_weights = model.get_layer("output_dense").get_weights()
    new_model.add(Dense(output_dim, name="output_dense"))
    new_model.get_layer("output_dense").set_weights(old_weights)

    # Set the old weights
    for i, l in enumerate(model.layers[:-1]):
        if "dense" in l.name:
            old_weights = model.get_layer("dense_{}".format(i)).get_weights()
            new_model.get_layer('dense_{}'.format(i)).set_weights(old_weights)
        elif "activation" in l.name:
            old_weights = model.get_layer("activation_{}".format(i)).get_weights()
            new_model.get_layer('activation_{}'.format(i)).set_weights(old_weights)
 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model

In [8]:
nmodel = extend(model, hidden_dim, output_dim)
nmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_0 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
activation_1 (PReLU)         (None, 100)               1.0       
_________________________________________________________________
output_dense (Dense)         (None, 10)                1010      
Total params: 89,611
Trainable params: 89,611
Non-trainable params: 0
_________________________________________________________________


In [56]:
# ------------------------------------------

In [ ]:
model = WeddingCake(784, )

In [1]:
len([5])

1

In [10]:
l = nmodel.get_layer('activation_1')

In [11]:
l.get_weights()

[-1.0]